### Практическая работа к уроку №2

#### 2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)
#### 3. Повторить п.2, но используя уже не медиану, а max

In [12]:
import pandas as pd
import numpy as np
import re
#предобработка текстов
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize
from razdel import tokenize # https://github.com/natasha/razdel
import pymorphy2
from gensim.corpora.dictionary import Dictionary

In [2]:
# Новости
news = pd.read_csv("articles.csv")
print(news.shape)
display(news.head(3))
# Пользователи и списки последних прочитанных новостей
users = pd.read_csv("users_articles.csv")
display(users.head(3))

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


### Получение векторного представления новостей

In [3]:
stopword_ru = stopwords.words('russian')
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [7]:
def clean_text(text):
    '''
    очистка текста
    на выходе очищеный текст
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}
morph = pymorphy2.MorphAnalyzer()

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова
    на выходе лист отлемматизированых токенов
    '''
    # [0]
    if not isinstance(text, str):
        text = str(text)
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [5]:
%%time
# Очистка текста
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

<ipython-input-4-35001d1a28bc>:13: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


CPU times: user 34.7 s, sys: 707 ms, total: 35.4 s
Wall time: 35.5 s


In [8]:
%%time
# Лемматизация текста
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

CPU times: user 4min 55s, sys: 355 ms, total: 4min 55s
Wall time: 4min 56s


In [13]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [28]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

CPU times: user 1min 26s, sys: 1.42 s, total: 1min 28s
Wall time: 1min 4s


In [29]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [31]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: пострадать тело мужчина египет дом жертва вирус
topic_1: гражданин nn век место фронт народный памятник
topic_2: рубль санкция миссия размер закон выплата торговый
topic_3: ракета дональд прогнозировать сообщество эволюция орудие океан
topic_4: научный исследование nn остров экипаж данные высота
topic_5: мозг модернизация взрывной рисунок вино лестница скот
topic_6: погибнуть операция произойти турция турецкий подчеркнуть чёрный
topic_7: фильм сократиться век конструкция выходной разделять сосед
topic_8: журнал планета выяснить квартира всё двигатель болезнь
topic_9: фонд новый всё центр первый станция проект
topic_10: фестиваль мышь журнал лётчик бежать пилот рекомендовать
topic_11: млн цена составить стоимость налог тыс составлять
topic_12: доллар снизить кризис подчёркивать сектор прогноз оплата
topic_13: белый девочка лекарство зеландия развиться свет чай
topic_14: россия nn млрд рост москва большой новый
topic_15: проверка nn белоруссия состояться организатор расширить па

In [32]:
# Векторное представление новости
def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [34]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.166411,0.0,0.062503,0.000000,0.0,0.0,0.012172,0.0,0.0,0.0
1,4896,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.385541,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
2,4897,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.123303,...,0.288182,0.0,0.137815,0.344737,0.0,0.0,0.041501,0.0,0.0,0.0
3,4898,0.0,0.0,0.0,0.0,0.334117,0.0,0.0,0.0,0.284707,...,0.077825,0.0,0.174708,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
4,4899,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.077390,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0


### Получение векторного представления пользователей

In [36]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [79]:
def get_user_embedding(user_articles_list, type_avg):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    if type_avg == "mean":
        user_vector = np.mean(user_vector, 0)
    elif type_avg == "median":
        user_vector = np.median(user_vector, 0)
    elif type_avg == "max":
        user_vector = np.max(user_vector, 0)
    return user_vector

In [80]:
def get_all_embedding(type_avg):
    # получим эмбединги для всех пользователей
    user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, type_avg), 1)])
    user_embeddings.columns = ['topic_{}'.format(i) for i in range(25)]
    user_embeddings['uid'] = users['uid'].values
    user_embeddings = user_embeddings[['uid']+['topic_{}'.format(i) for i in range(25)]]
    return user_embeddings

### Обучим модель на конкретной downstream-задаче

In [56]:
target = pd.read_csv("users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [81]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import (f1_score, roc_auc_score,
                            precision_score, classification_report,
                            precision_recall_curve, confusion_matrix)

import matplotlib.pyplot as plt
%matplotlib inline

In [82]:
def train_model(type_avg):
    user_embeddings = get_all_embedding(type_avg)
    
    X = pd.merge(user_embeddings, target, 'left')
    X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                        X['churn'], random_state=0)
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    
    #наши прогнозы для тестовой выборки
    preds = logreg.predict_proba(X_test)[:, 1]

    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    roc_auc = roc_auc_score(y_test, preds)
    
    return fscore[ix], precision[ix], recall[ix], roc_auc

In [84]:
results = {}

for type_avg in ['mean', 'median', 'max']:
    metrics = train_model(type_avg)
    results[type_avg] = metrics

In [90]:
df = pd.DataFrame.from_dict(results, orient='index',
                            columns=['fscore', 'precision', 'recall', 'roc_auc']).round(3)
df

,fscore,precision,recall,roc_auc
mean,0.671,0.594,0.771,0.948
median,0.765,0.696,0.849,0.971
max,0.837,0.858,0.816,0.981


#### 3. Повторить процесс обучения, но уже взвешивая новости по tfidf

In [167]:
# с помощью common_dictionary.dfs узнаем в скольких новостях встречается каждое слово из словаря
# Пересчитаем корпус заменив частоту встречания слова в новости на idf
import math

len_dict = len(common_dictionary)
idf_corpus = []

for doc in common_corpus:
    news_corpus = []
    for word, freq in doc:
        idf = math.log(len_dict / common_dictionary.dfs[word])
        news_corpus.append((word, idf))
    idf_corpus.append(news_corpus)

In [ ]:
# Повторим все те же этапы обучения, но с новым корпусом
%%time
lda = LdaModel(idf_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]

doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))


In [179]:
results_idf = {}

for type_avg in ['mean', 'median', 'max']:
    metrics = train_model(type_avg)
    results_idf[f"idf_{type_avg}"] = metrics

In [181]:
df_idf = pd.DataFrame.from_dict(results_idf, orient='index',
                            columns=['fscore', 'precision', 'recall', 'roc_auc']).round(3)
df_idf

,fscore,precision,recall,roc_auc
idf_mean,0.695,0.609,0.808,0.944
idf_median,0.721,0.674,0.776,0.957
idf_max,0.729,0.763,0.698,0.960


#### 5. Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score

In [189]:
union_df = pd.concat([df, df_idf]).sort_values(by=['roc_auc'], ascending=False)
union_df

,fscore,precision,recall,roc_auc
max,0.837,0.858,0.816,0.981
median,0.765,0.696,0.849,0.971
idf_max,0.729,0.763,0.698,0.960
idf_median,0.721,0.674,0.776,0.957
mean,0.671,0.594,0.771,0.948
idf_mean,0.695,0.609,0.808,0.944


#### 6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

Скорей всего, максимизация по всем новостям, которые прочитал пользователь, вероятностей принадлежности новости к той или иной тематике, оказалась эффективней всего, потому что таким образом мы даем больше информации для модели, придавая больший вес определенным тематикам.

Медиана оказалась лучше среднего, потому что она не подвержена выбросам, ведь пользователь мог единожды прочесть новость на определенную тематику, но данная тематика новостей не интересует его больше всего.

По поводу того, что взвешивание новостей по количеству слов в новости оказалось лучшем чем взвешивание с помощью idf идей нет, к сожалению
